## Test

## Initialization

In [24]:
import os
import sys
import json
import openai

openai_config = json.load(open("openai.json", "r+"))
openai.api_key = openai_config["api_key"]
openai.organization = openai_config["organization_id"]

In [ ]:
model_list = openai.Model.list()
print("model_list available:")
for model in model_list["data"]:
    print(model["id"])

## Interface

In [71]:
import textwrap

def wrap(str, initial_indent="", indent=""):
    return ("\n" + indent).join(textwrap.wrap(str, initial_indent=initial_indent, width=100))

# print(wrap("Hello world. Hello world. Hello world. Hello world. Hello world. Hello world. Hello world. Hello world. Hello world. Hello world. Hello world. Hello world. Hello world. Hello world. Hello world. Hello world. Hello world. Hello world. Hello world. Hello world. Hello world. ", indent="    "))

In [96]:
class Conversation:
    # messages: List<{role: String, content: String}>
    def __init__(self, messages=None):
        if messages is None:
            self.messages = []
        else:
            self.messages = messages

    def load(filename):
        return Conversation(messages=json.load(open(filename, mode="r+")))
    
    def save(self, filename):
        json.dump(self.messages, open(filename, mode="w+"))

    # roles: user, system, assistant
    def message(self, role, content):
        self.messages.append({'role': role, 'content': content})

    def print(self):
        for message in self.messages:
            print(f"# ROLE:{message['role']}")
            print(f"{message['content']}\n")

    def query(self):
        print("[>] query sent ...")
        result = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=self.messages
        )
        print("[>] query answered ✓")
        try:
            choice = result['choices'][0]
        except IndexError:
            print("response with node choices")

        if choice['finish_reason'] != "stop":
            raise Exception(f"finish reason: {choice['finish_reason']}")

        print("[>] query is valid ✓")
        message = choice['message']
        self.messages.append(message)

        print(40 * "=")
        print(message['content'])
        print(40 * "=")


## Experiments

In [ ]:
# test1: simple conversation
convo = Conversation.load("conversations/test/test1.json")
convo.print()

In [105]:
# test2: simple template

def shortstory(name, author, genre, theme, main_idea):
    convo = Conversation()
    convo.message("system", "You are a creative and knowledgeable writing assistant for a short story author. The author wants to write innovative, unique, original stories that appeal to niche audiences and not necessarily to the general public.")
    convo.message("user",
                  f"I am writing a {genre} short story with this theme: {theme}. Write a list of interesting, original, and thematic ideas that would be interesting to include in my short story.")
    convo.query()
    convo.message("user",
                  "Write a brief summary of a short story that ties together a few of the ideas that you listed above. The summary should include a description of the setting, an introduction to the main characters, the main plot points, and a conclusion.")
    convo.query()
    convo.message("user",
                  f"Write a revised version of that summary which uses most of the same details, but also makes sure to include this main idea: {main_idea}.")
    convo.query()
    convo.message("user",
                  f"Write a short story in the style of {author} and based on that summary. Make sure the story is around 2500 words. The story should first introduce the setting and main characters, then flush out the details of and transition smoothly between all the plot points, and then finish the story with the conclusion.")
    convo.query()
    convo.message("user",
                  "Write a revised version of that short story. Make the writing tighter, flush out the setting and characters with more unique details, explain more about why each event happens, and include some dialogue. Make the revised story under 2000 words.")
    convo.query()
    convo.save(f"conversations/shortstory/{name}.json")


name = "story3"

shortstory(
    name,
    author="Isaac Asimov",
    genre="sci-fi",
    theme="humans explore a planet inhabited by intelligent and hyper-industrialized aliens with advanced technology",
    main_idea="the aliens build cities and have a religion that involves very strange beliefs about the universe, which results in a complicated conflict between the humans and the aliens"
)


[>] query sent ...
[>] query answered ✓
[>] query is valid ✓
Sure, here are some interesting and original thematic ideas that you could include in your sci-fi short story set on an alien planet:

1. Cultural barriers: The humans have a hard time understanding the culture and social norms of the aliens, leading to miscommunications and conflicts.

2. Environmental dangers: The planet is full of dangerous phenomena such as extreme weather, hostile terrain, and unpredictable wildlife, which presents a challenge for the humans as they explore the planet.

3. Technological marvels: The aliens have technology that is vastly superior to that of the humans, including advanced energy sources, powerful weapons, and faster-than-light travel capabilities.

4. Societal disparities: The aliens live in a society that is divided by socioeconomic class, with the wealthy upper class enjoying luxurious lifestyles while the poor struggle to make ends meet.

5. Legal systems: The aliens have a unique legal